In [24]:
import pandas as pd 

In [25]:
import numpy as np

In [26]:
!ls ../out

output_CM_global.csv	output_SLATM_global.csv
output_FCHL_global.csv	output_SOAP_global.csv


In [27]:
slatm_out = pd.read_csv("../out/output_CM_global.csv", index_col=0)

In [28]:
slatm_out

,SolN,Fragments,ObjValNoPen,ObjValWithPen
0,1,"['frag_04_c00001', 'frag_16_c00001']",25789.307767,25789.307767
1,2,"['frag_04_c00001', 'frag_15_c00001']",25819.650390,25819.650390
2,3,"['frag_06_c00001', 'frag_13_c00001']",25847.403029,25847.403029
3,4,"['frag_07_c00001', 'frag_12_c00001']",25849.587220,25849.587220
4,5,"['frag_07_c00001', 'frag_11_c00001']",25855.206498,25855.206498
...,...,...,...,...
995,996,"['frag_01_c00001', 'frag_03_c00001', 'frag_05_...",59797.586518,99797.586518
996,997,"['frag_07_c00001', 'frag_08_c00001', 'frag_15_...",39817.363881,99817.363881
997,998,"['frag_07_c00001', 'frag_08_c00001', 'frag_16_...",39849.082916,99849.082916
998,999,"['frag_01_c00001', 'frag_03_c00001', 'frag_05_...",59868.311162,99868.311162


In [29]:
from ast import literal_eval 

In [30]:
slatm_out["Fragments"] = slatm_out["Fragments"].apply(literal_eval)

In [31]:
# excess is missing Hs - to fix 

In [32]:
import qml

In [33]:
qm9_mol = qml.Compound("../targets/qm9_0.xyz")
qm9_ncharges = qm9_mol.nuclear_charges

In [34]:
qm9_ncharges

array([8, 6, 6, 7, 6, 8, 8, 7, 6, 1, 1])

In [35]:
def count_ncharges(list_ncharges, elements=[1,6,7,8,16]):
    ncharge_count = np.zeros(len(elements))
    for ncharge in list_ncharges:
        for i, elem in enumerate(elements):
            if ncharge == elem:
                ncharge_count[i] += 1
    return ncharge_count

In [36]:
qm9_counts = count_ncharges(qm9_ncharges)

In [37]:
qm9_counts

array([2., 4., 2., 3., 0.])

In [38]:
def get_excess_frags(frag_list):
    mols = [qml.Compound("../amons-qm9/"+x+".xyz") for x in frag_list]
    ncharges = np.concatenate([mol.nuclear_charges for mol in mols])
    ncharge_count = count_ncharges(ncharges)
    
    diff_count = ncharge_count - qm9_counts
    diff_count = np.array(diff_count).astype(int)
    excess = []
    for i, count in enumerate(diff_count): 
        ncharge = [1,6,7,8,16][i]
        if count > 0 :
            for i in range(count):
                excess.append(ncharge)
        elif count < 0:
            for i in range(abs(count)):
                excess.append(-ncharge)
    return excess

In [39]:
slatm_out["Excess"] = slatm_out["Fragments"].apply(get_excess_frags)

In [40]:
slatm_out

,SolN,Fragments,ObjValNoPen,ObjValWithPen,Excess
0,1,"[frag_04_c00001, frag_16_c00001]",25789.307767,25789.307767,"[1, 1]"
1,2,"[frag_04_c00001, frag_15_c00001]",25819.650390,25819.650390,"[1, 1, 1, -7, 8]"
2,3,"[frag_06_c00001, frag_13_c00001]",25847.403029,25847.403029,"[1, 1, 1, 1, 1, 7, -8]"
3,4,"[frag_07_c00001, frag_12_c00001]",25849.587220,25849.587220,"[1, 1, 1, 1, 1, 7, -8]"
4,5,"[frag_07_c00001, frag_11_c00001]",25855.206498,25855.206498,"[1, 1, 1, 1, 1, 7, -8]"
...,...,...,...,...,...
995,996,"[frag_01_c00001, frag_03_c00001, frag_05_c0000...",59797.586518,99797.586518,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 7, 8, 8]"
996,997,"[frag_07_c00001, frag_08_c00001, frag_15_c00001]",39817.363881,99817.363881,"[1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 7, 7, 8]"
997,998,"[frag_07_c00001, frag_08_c00001, frag_16_c00001]",39849.082916,99849.082916,"[1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 7, 7, 7]"
998,999,"[frag_01_c00001, frag_03_c00001, frag_05_c0000...",59868.311162,99868.311162,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 8, 8]"


In [41]:
slatm_out.sort_values(by="ObjValWithPen").iloc[0:30]

,SolN,Fragments,ObjValNoPen,ObjValWithPen,Excess
0,1,"[frag_04_c00001, frag_16_c00001]",25789.307767,25789.307767,"[1, 1]"
1,2,"[frag_04_c00001, frag_15_c00001]",25819.650390,25819.650390,"[1, 1, 1, -7, 8]"
2,3,"[frag_06_c00001, frag_13_c00001]",25847.403029,25847.403029,"[1, 1, 1, 1, 1, 7, -8]"
3,4,"[frag_07_c00001, frag_12_c00001]",25849.587220,25849.587220,"[1, 1, 1, 1, 1, 7, -8]"
4,5,"[frag_07_c00001, frag_11_c00001]",25855.206498,25855.206498,"[1, 1, 1, 1, 1, 7, -8]"
5,6,"[frag_03_c00001, frag_16_c00001]",25918.671941,25918.671941,"[1, 1, 1, 7, -8]"
6,7,"[frag_03_c00001, frag_15_c00001]",25966.674231,25966.674231,"[1, 1, 1, 1]"
7,8,"[frag_09_c00001, frag_11_c00001]",26754.740868,26754.740868,"[1, 1, 1, 1, 1, 6, -7]"
8,9,"[frag_10_c00001, frag_11_c00001]",26764.480169,26764.480169,"[1, 1, 1, 1]"
9,10,"[frag_09_c00001, frag_12_c00001]",26805.732416,26805.732416,"[1, 1, 1, 1, 1, 6, -7]"
